## A test space for a new *solar system object* metric ##

Solar system metrics are slightly different from standard metrics, in that they are intended to evaluate the observations of a given object instead of the exposures on the sky. 

The first step in evaluating solar system metrics is to create the expected observations of a given population; this is done using the `rubin_sim.movingObjects` module. A high-level command to do this is: 
```makeLSSTobs``` (from `rubin_sim/bin/movingObjects`)
and its associated flags/kwargs.
A typical invocation using the reasonable defaults would look like:
```
makeLSSTobs --opsimDb OPSIMDB --orbitFile ORBITFILE --outDir OUTDIR
```

With these observations in-hand, development of your metric can commence. It is reasonable to only create observations for a few orbits for preliminary testing.


In [1]:
# Some modules you're likely to want .. add whatever is needed.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline   
# %matplotlib notebook

In [2]:
# Import MAF
import rubin_sim.maf as maf

You need to connect to the opsim output, so let's do that first. <br>
It's easy to use the current baseline simulation included with $RUBIN_SIM_DATA_DIR, so let's start with that.

In [3]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)

/Users/lynnej/rubin_sim_data/sim_baseline/baseline_nexp2_v1.7_10yrs.db
baseline_nexp2_v1.7_10yrs


In [4]:
# Connect to the database so we can use it with MAF.
opsim_db = maf.OpsimDatabase(opsim_fname)

And let's set up a slicer that will give us the observations that overlap a single point on the sky.

In [5]:
# Specify ra / dec of the point we want to work with on the sky - in degrees. 
# (these can be lists, if you want to work on multiple, limited points on the sky)
test_ra = 0.0
test_dec = -20.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

Now set up to work with our Metric. Remember that the Metric will work on a single DataSlice at a time -- so 
*all* of (and *only*) the observation information it receives will be the visits relevant to this test_ra/test_dec location.

In [16]:
class MyMetricInProgress(maf.BaseMetric):
    """Documentation please. Numpy style docstrings. 
    
    This example metric just finds the time of first observation of a particular part of the sky.
    
    Parameters
    ----------
    specificColumns : `str`, opt
        It's nice to be flexible about what the relevant columns are called, so specify them here.
        seeingCol = FWHMeff, etc. 
    kwargs : `float`, ? 
        Probably there are other things you need to set?
    """
    def __init__(self, mjdCol='observationStartMJD', **kwargs):
        self.mjdCol = mjdCol
        cols = [self.mjdCol, ] # Add any columns that your metric needs to run -- mjdCol is just an example
        super().__init__(col=cols, units='#', **kwargs)
        
    def run(self, dataSlice, slicePoint=None):
        # This is where you write what your metric does. 
        # dataSlice == the numpy recarray containing the pointing information, 
        # with the columns that you said you needed in 'cols'
        # slicePoint == the information about where you're evaluating this on the sky -- ra/dec,
        # and if you specified that you need a dustmap or stellar density map, etc., those values will also 
        # be defined here
        
        # here's a super simple example .. replace with your own code to calculate your metric values
        tMin = dataSlice[self.mjdCol].min()
        return tMin
    
# When you re-run this cell, you may get a warning that the metric name already exists - that's ok! 

/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/maf/metrics/baseMetric.py:36: UserWarning: Redefining metric __main__.MyMetricInProgress! (there are >1 metrics with the same name)
  warnings.warn('Redefining metric %s! (there are >1 metrics with the same name)' % (metricname))


The typical way to use this metric with a slicer within MAF would be as follows: 

In [7]:
# Set up the metric
mymetric = MyMetricInProgress()

In [8]:
# Define a sqlconstraint, if we need to just use a (large) subset of the opsim visits
sqlconstraint = None   # no constraint, make all visits available

# Examples of other potentially useful sqlconstraints:
# sqlconstraint = 'filter = "r"'  # just select the visits in a particular filter
# sqlconstraint = 'note not like "%DD%"'  # don't choose any of the DD field visits
# sqlconstraint = 'night < 365'  # only use visits in the first year of the survey

In [9]:
# We already defined the slicer - combine the metric, slicer and sqlconstraint in a MetricBundle:
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)

In [10]:
# Pass the bundle (along with any other bundles to be run on this opsim) to a MetricBundleGroup in order to
# calculate the metric bundle values.
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)
# And calculate the metric
g.runAll()

Querying database SummaryAllProps with no constraint for columns ['rotSkyPos', 'observationStartMJD', 'fieldRA', 'fieldDec'].
Found 2045493 visits
Running:  ['test_metric']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


And then you can look at the `bundle.metricValues` to see what your metric calculated and how well things worked.

In [11]:
bundle.metricValues

masked_array(data=[59857.144187715814],
             mask=[False],
       fill_value=-666.0)

BUT, when you're testing a new metric, you might run this many times over .. and querying the database each time is not necessary, if you are re-using the same columns from the database. If you're re-using exactly the same data (same columns, same sqlconstraint), you can skip the query like this:

In [12]:
# g.simData is the simulation visit data that the previous MetricBundleGroup queried from the database
g.simData[0:2]

array([(188.75064607, 59853.98564424, 298.33025759,  0.73493423),
       (194.09142482, 59853.98609138, 295.86447427, -1.05483733)],
      dtype=(numpy.record, [('rotSkyPos', '<f8'), ('observationStartMJD', '<f8'), ('fieldRA', '<f8'), ('fieldDec', '<f8')]))

In [17]:
# redefine your metric and rerun the cell where it was defined (above) - 
#  I'll swap the minimum time to the maximum, for this example, just so you can see the result changed 
# and then set up a new metric object:
mymetric = MyMetricInProgress()  # version X
# and set up a new MetricBundle object 
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)

# Then set up a NEW and DIFFERENTLY NAMED MetricBundleGroup
g2 = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)

In [18]:
# But then run it like this: 
g2.runCurrent(simData=g.simData, constraint=sqlconstraint)

Running:  ['test_metric']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [19]:
# See new bundle metric values
bundle.metricValues

masked_array(data=[63504.08757648483],
             mask=[False],
       fill_value=-666.0)